In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/My Drive/Colab Notebooks/IFT2015/")
import sys
sys.path.append("drive/My Drive/Colab Notebooks/IFT2015/")
#check if you are in the right directory
!ls

In [1]:
import gzip
from itertools import islice
from graph_yq import DeBrujinGraph

In [2]:
def read_fastq(path):
    with gzip.open(path, 'rt') as f:
        for line in f:
            seqid, description = line[1:].rstrip().split(maxsplit=1)
            sequence = f.readline().rstrip()
            _ = f.readline()
            quality = f.readline().rstrip()
            yield seqid, description, sequence, quality

In [3]:
k = 21
kmers = []

In [4]:
%%time
for _, _, seq, _ in read_fastq('reads.fastq.gz'):
    kmers.extend([seq[i:i+k] for i in range(len(seq) - k + 1)])

CPU times: user 1.6 s, sys: 164 ms, total: 1.77 s
Wall time: 1.84 s


In [5]:
%%time
length = min(10000, len(kmers))
graph = DeBrujinGraph(nodes = kmers[:length], k = 21)

CPU times: user 4.42 s, sys: 23.6 ms, total: 4.44 s
Wall time: 4.44 s


In [6]:
print("node counts:", graph.node_count())
print("edge counts:", graph.edge_count())

node counts: 10000
in couting edges methods
edge counts: 9875


## Question 3 Parcour

In [7]:
def DFS( g, u, discovered ):
    """deep first search of a graph.
    params
        g: a graph
        u: a node of graph g
        discovered: a dictionary marking where a edge is visited
    return
        None, discovered modified
    """
    for v in g.successors( u ):
        if v not in discovered:
            discovered[v] = u # u represents parent node of v
                              # v carries the edge information v[-1]
            DFS( g, v, discovered )
            
            
def build_fragment( g, u, v, discovered ):
    """build a fragment from node u to v based on the discovered paths in graph
    params
        g: graph
        u: start node of a fragment, str
        v: end node of a fragment, str
        discovered: dictionary from a deep first search of a graph g
    returns
        fragment: str
    """
    frag = []
    if v in discovered:
        walk = v
        while walk is not u:
            parent = discovered[walk]
            frag.append(walk[-1])
            walk = parent
        # walk is now u
        
    frag.reverse()
    frag = u + "".join(frag)
    return frag

In [8]:
# find the nodes who have no predecessors
def find_nodes(g, has_no_predecessors = True):
    """find all nodes without predecessors or without successors controled by
    a bool parameter
    params
        g: graph
        has_no_predecessors: bool, if True, find all nodes without predecessors,
            If False, find all nodes without successors
    returns
        results, list of nodes
    """
    results = []
    for node in g.nodes():
        func = g.predecessors
        if has_no_predecessors == False:
            func = g.successors
        if len(func(node)) == 0: # no predecessors
            results.append(node)
    return results           

In [9]:
%%time
nodes_without_predecessors = find_nodes(graph)
nodes_without_successors = find_nodes(graph, False)

CPU times: user 4.21 s, sys: 0 ns, total: 4.21 s
Wall time: 4.2 s


In [10]:
print(nodes_without_predecessors)
print(len(nodes_without_predecessors))

['CAACACCTCTCTCCTGAACCT', 'GTTAGCACTTGGAAGACAATT', 'ACCACTTCTACTCCAGACGTC', 'GATAATAAAAACGACCCCTGC', 'TGCAGCCTACGAAATGACTCC', 'TACAAGAAGCTCTTCATTTTG', 'ACTTCGAAAGATCATATGGAT', 'GAGGGTTGCCAATTCGTTTCA', 'TCGAATTCATAATTTCTGTAA', 'AAGTCACGAATTGGCCCGTAC', 'GAGACTACATCATCCCATGAT', 'TGCTACACATCGAGAATCATA', 'TGGGTACGGAAGCTATGGTGG', 'CCAAATCTTCGAATGTTGACT', 'TCACAGAGATGGAACTTATGC', 'GTGAAGTTGAGAGGAGGAGAT', 'TTACTACCCATCGGTTAATAA', 'CCGAGTCCGATCATCACTGGG', 'AAGATTGTGCTGGATAATATT', 'CGTTGAAGACGAAATTGTCAA', 'CCAGCTCTTGAATCATCTGCG', 'CCAACTTTGCCGTTGGTGTTG', 'GCTGCTGCTTGCGTCTTGGGT', 'CGACAGAGTTTATAACTCGAA', 'TCGGAATGTATCTCGTGATTT', 'AGCTCCCAGAAGTTGCAAGAG', 'ACTTGTCTACACGAAAACCTT', 'CGTTGAGAGATTTGGTGGCTG', 'AGAACCATTCTCGGAGGACAT', 'CAGGACTTTAGCAAATTCAAT', 'AAAGTCGTGACCGATGCAAAG', 'CGGCAACCTCCCTATCGCCAA', 'TTCCAAGCGAAGCGGGGATTG', 'AAAAGTCAACTGCCAAAAGGA', 'ATTGCTGCATTTTGGCCTCAA', 'CGGAAGACCACGCAAACCGAC', 'GCTTTTCCGTTTAAATGTGCA', 'CTTGGATGCATTCATAGTAAA', 'AAAAAGTCGGATATGGCAAAA', 'TCATCCCAACAAGTGAAGAAT',

In [11]:
print(nodes_without_successors)
print(len(nodes_without_successors))

['TCCGAGTGTGCACAGCATCAT', 'CAATATACACTGGATTCATGT', 'CGGCGAAATCACAAATTCTGC', 'GATTGATGGCTCAAATTCCAA', 'AGTTAAACACCCAAAAATGGT', 'AGCATAACAGATACTTTGTCG', 'GAAACTCTGCTCACGGGCGAT', 'AGAAAATCAACGAATACATGA', 'TCATACTTAATCGATGGAACT', 'AAAAGACAGTCACGAATGATC', 'TGATGGATGAAGTTATTCGAG', 'TTGTCGAAGGAATGCCAGCAT', 'TTCATGCTCAGTTGATGAAAG', 'AACTCCCCAGCAGCAGTTAAA', 'AGTGTGTGTTTAGTACTGGGA', 'TCAATCAGAACTTGATTCGAG', 'CAAGTGGTTGTTTCAAAATAG', 'AAGACCCCGACAATGTGAAAG', 'ACGTTGTCGACTACGCCACGG', 'GGGCGGTGATTAAGAAAAAGG', 'TGGATGTCCAATTGGAGATGT', 'TGAAAGTGTATCAGAAGAAAG', 'GTGCATTAAAATCGCTACTCG', 'AAACAACAGAACTCCAGTCAA', 'GAGTGATTAAAATGCTCATTG', 'TCCGAAAACCAAACAGCACAG', 'GAACAACTCGTCTCGGAGATT', 'GAGCACGTAAAAGGAAGTCCA', 'CCGGGGGAGCAGTTTGGGTAT', 'CGATAACCCAGAAGTCACCGA', 'AAGTATACACATGTCCGAGGG', 'ATATGGAATTGCAACATATCG', 'CTCACTGCTGAAGAAACACCT', 'TCAAAGGAGCAAAAGAGGCTC', 'ACTAGTAATTTGGTGGGCAGA', 'AGATTACTTCCCATCAAATTC', 'TTGCTCGCCAAGCTGGTGGTG', 'AACAAATCAAGACTTTAATGA', 'AAAACTATTGGAGATCATTCT', 'GCCGAGGACTTCTAGTGGATG',

In [12]:
fragments = []
for origin_node in nodes_without_predecessors:
    discovered = {origin_node: '-'}
    DFS(graph, origin_node, discovered)
    for dest_node in nodes_without_successors:
        if discovered.get(dest_node, None) is not None:
            fragment = build_fragment(graph, origin_node, dest_node, discovered)
            fragments.append(fragment)

In [13]:
print(len(fragments))

125


In [14]:
print(fragments[1])

GTTAGCACTTGGAAGACAATTGGAGAGTATTTTGACTCCACCTGAGCGTCGCCTATTCTTTAGAACTTCTGCTGAACACATGGCTAGAGGCTGTCCAATG


In [15]:
# Test

seq = "ACTGAGTCATGGATG"
k = 4
kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
print("seq: ", seq)
print("kmers: ", kmers)

print("initialize graph with kmers and k")
graph2 = DeBrujinGraph(kmers, k = k)
print(graph2)

seq:  ACTGAGTCATGGATG
kmers:  ['ACTG', 'CTGA', 'TGAG', 'GAGT', 'AGTC', 'GTCA', 'TCAT', 'CATG', 'ATGG', 'TGGA', 'GGAT', 'GATG']
initialize graph with kmers and k
in couting edges methods
G( 12 nodes{ TGGA ACTG CATG CTGA GATG TCAT GGAT AGTC ATGG GAGT TGAG GTCA }, 12 edges{ ('TGGA', 'GGAT') ('ACTG', 'CTGA') ('CATG', 'ATGG') ('CTGA', 'TGAG') ('GATG', 'ATGG') ('TCAT', 'CATG') ('GGAT', 'GATG') ('AGTC', 'GTCA') ('ATGG', 'TGGA') ('GAGT', 'AGTC') ('TGAG', 'GAGT') ('GTCA', 'TCAT') } )


In [16]:
start_nodes = find_nodes(graph2)
end_nodes = find_nodes(graph2, False)

In [17]:
print(start_nodes)
print(end_nodes)

['ACTG']
[]


In [18]:
def read_fasta(path):
    with gzip.open(path, 'rt') as f:
        accession, description, seq = None, None, None
        for line in f:
            if line[0] == '>':
                # yield current record
                if accession is not None:
                    yield accession, description, seq
                    
                # start a new record
                accession, description = line[1:].rstrip().split(maxsplit=1)
                seq = ''
            else:
                seq += line.rstrip()